## Notebook 1: Data Scraping 

### 1.Internal Features of Movie without collection/series
##### 1.1 Scraping Code definition

In [1]:
def get_movie_url_and_basic_info(page_link,csv_number):
    import requests
    from bs4 import BeautifulSoup

    response = requests.get(page_link)
    movie_url_list = []
    runtime = None
    genre = None
    source_page = BeautifulSoup(response.content,'lxml')
    movie_abstract = source_page.find_all('div',class_='lister-item-content')
    for movie_abstract in movie_abstract:
        movie_url = 'https://www.imdb.com' + movie_abstract.find('a').get('href')
        movie_name = movie_abstract.find('a').get_text()
        if movie_abstract.find('span', class_='runtime'):
            if ',' not in movie_abstract.find('span', class_='runtime').get_text()[:-4]:
                runtime = int(movie_abstract.find('span', class_='runtime').get_text()[:-4])
            else:
                runtime = int(movie_abstract.find('span', class_='runtime').get_text()[:-4].replace(',',''))
        else:
            runtime = runtime
        if movie_abstract.find('span', class_='genre'):
            genre = movie_abstract.find('span', class_='genre').get_text().strip().split(', ')
        movie_url_list.append([movie_name,runtime,genre,movie_url])
    for i in range(250,csv_number,250): 
        next_page_link = '&start=' + str(i+1) + '&ref_=adv_nxt'
        new_page_link = page_link + next_page_link
        next_response = requests.get(new_page_link)
        next_page = BeautifulSoup(next_response.content,'lxml')
        movie_abstract = next_page.find_all('div',class_='lister-item-content')
        for movie_abstract in movie_abstract:
            movie_url = 'https://www.imdb.com' + movie_abstract.find('a').get('href')
            movie_name = movie_abstract.find('a').get_text()
            if movie_abstract.find('span', class_='runtime'):
                if ',' not in movie_abstract.find('span', class_='runtime').get_text()[:-4]:
                    runtime = int(movie_abstract.find('span', class_='runtime').get_text()[:-4])
                else:
                    runtime = int(movie_abstract.find('span', class_='runtime').get_text()[:-4].replace(',',''))
            else:
                runtime = runtime
            if movie_abstract.find('span', class_='genre'):
                genre = movie_abstract.find('span', class_='genre').get_text().strip().split(', ')
            movie_url_list.append([movie_name,runtime,genre,movie_url])
    return movie_url_list

In [2]:
def get_titleBar_info(movie_page):
    titleBar_info = movie_page.find('div', class_='title_block')
    rate = None
    rating_count = None
    if titleBar_info.find('div', class_='ratingValue'):
        rate = float(titleBar_info.find('div', class_='ratingValue').get_text().strip()[:-3])
    if titleBar_info.find('span', class_='small', itemprop='ratingCount'):
        count_string = titleBar_info.find('span', class_='small', itemprop='ratingCount').get_text()
        rating_count = ''
        for num in count_string:
            if num in '0123456789':
                rating_count += num
        rating_count = int(rating_count)
    return rate,rating_count

In [3]:
def get_titleDetails_info(movie_page,movie_id):
    from datetime import datetime
    import requests
    from bs4 import BeautifulSoup
    import re

    titleDetails_info = movie_page.find('div', class_='article', id='titleDetails')
    country,language,release_date,original_title = None,None,None,None
    budget,gross_usa,cumulative_worldwide_gross = None,None,None
    production_company = []
    if titleDetails_info:
        text_block = titleDetails_info.find_all('div',class_='txt-block')
        for block_detail in text_block:
            if block_detail.find('h4',class_='inline'):
                if block_detail.find('h4',class_='inline').get_text()=='Country:':
                    countrys = block_detail.find_all('a')
                    country = [i.get_text() for i in countrys]
                if block_detail.find('h4',class_='inline').get_text()=='Language:':
                    languages = block_detail.find_all('a')
                    language = [i.get_text() for i in languages]
                if block_detail.find('h4',class_='inline').get_text()=='Release Date:':
                    try:
                        pattern = r'(\w{1,2} )(\w+ )(\d{4})'
                        match = re.search(pattern,block_detail.get_text().strip('\n'))
                        group = match.groups()
                        date = ''
                        for i in group:
                            date += i
                        release_date = datetime.strptime(date, '%d %B %Y').date()
                    except:
                        continue
                if block_detail.find('h4',class_='inline').get_text()=='Also Known As:':
                    raw_title = block_detail.get_text().strip().split('\n')
                    for i in raw_title:
                        if 'Also Known As:' in i:
                            original_title = i[15:]
                if block_detail.find('h4',class_='inline').get_text()=='Budget:':
                    budget = ''
                    post_strip1 = block_detail.get_text().strip()
                    for i in post_strip1:
                        try:
                            int(i)
                            budget+=i
                        except:
                            continue
                if block_detail.find('h4',class_='inline').get_text()=='Gross USA:':
                    gross_usa = ''
                    post_strip2 = block_detail.get_text().strip()
                    for i in post_strip2:
                        try:
                            int(i)
                            gross_usa+=i
                        except:
                            continue
                if block_detail.find('h4',class_='inline').get_text()=='Cumulative Worldwide Gross:':
                    cumulative_worldwide_gross = ''
                    post_strip3 = block_detail.get_text().strip()
                    for i in post_strip3:
                        try:
                            int(i)
                            cumulative_worldwide_gross+=i
                        except:
                            continue
                if block_detail.find('h4',class_='inline').get_text()=='Production Co:':
                    companies = block_detail.find_all('a')
                    raw_production_company = [i.get_text() for i in companies]
                    if 'See more' in raw_production_company:
                        production_company_url = 'https://www.imdb.com/title/'+movie_id+'/companycredits?ref_=tt_dt_co'
                        production_company_page_request = requests.get(production_company_url)
                        production_company_page = BeautifulSoup(production_company_page_request.content,'lxml')
                        try:
                            company_list = production_company_page.find('ul', class_='simpleList').find_all('a')
                            production_company = [{i.get('href')[9:18]:i.get_text()} for i in company_list]
                        except:
                            continue
    return country,language,release_date,original_title,budget,gross_usa,cumulative_worldwide_gross,production_company

In [4]:
def get_cast_and_crew_info(movie_id):
    import requests
    from bs4 import BeautifulSoup

    director_list = []
    writer_list = []
    cast_list_odd,cast_list_even,cast_list = [],[],[]
    cast_page_url = 'https://www.imdb.com/title/'+movie_id+'/fullcredits?ref_=tt_cl_sm#cast'
    cast_page_response = requests.get(cast_page_url)
    cast_page = BeautifulSoup(cast_page_response.content,'lxml')
    labor_tables = cast_page.find('div',class_='header',id='fullcredits_content')
    try:
        h4 = labor_tables.find_all('h4',class_='dataHeaderWithBorder')
        table = labor_tables.find_all('table', class_='simpleTable simpleCreditsTable')
        for i in range(len(h4)):
            if 'Directed by' in h4[i].get_text():
                directors = table[i].find_all('a')
                for directors in directors:
                    director_dict = dict()
                    director_dict['name'] = directors.get_text().strip()
                    director_dict['id'] = directors.get('href')[6:15]
                    director_list.append(director_dict)
            if 'Writing Credits' in h4[i].get_text():
                writers = table[i].find_all('a')
                for writers in writers:
                    writer_dict = dict()
                    writer_dict['name'] = writers.get_text().strip()
                    writer_dict['id'] = writers.get('href')[6:15]
                    writer_list.append(writer_dict)
            if 'Cast' in h4[i].get_text():
                cast_tables = cast_page.find('table', class_='cast_list')
                odd_line = cast_tables.find_all('tr', class_='odd')
                even_line = cast_tables.find_all('tr', class_='even')
        for odd in odd_line:
            cast_dict = dict()
            info = odd.find_all('a')
            cast = info[1].get_text().strip()
            cast_id = info[1].get('href')[6:15]
            cast_dict['cast_id'] = cast_id
            cast_dict['name'] = cast
            if info[2:]:
                character = [i.get_text().strip() for i in info[2:]]
                cast_dict['character'] = character
                cast_list_odd.append(cast_dict)
            else:
                info1 = odd.find_all('td', class_ = 'character')
                a = ['/','\n']
                character = [i.get_text().strip().replace('\n','').replace('/','').replace(' ','') for i in info1]
                cast_dict['character'] = character
                cast_list_odd.append(cast_dict)
        for even in even_line:
            cast_dict = dict()
            info = even.find_all('a')
            cast = info[1].get_text().strip()
            cast_id = info[1].get('href')[6:15]
            cast_dict['cast_id'] = cast_id
            cast_dict['name'] = cast
            if info[2:]:
                character = [i.get_text().strip() for i in info[2:]]
                cast_dict['character'] = character
                cast_list_even.append(cast_dict)
            else:
                info1 = even.find_all('td', class_ = 'character')
                a = ['/','\n']
                character = [i.get_text().strip().replace('\n','').replace('/','').replace(' ','') for i in info1]
                cast_dict['character'] = character
                cast_list_even.append(cast_dict)
        for num in range(min(len(cast_list_odd),len(cast_list_even))):
            cast_list.append(cast_list_odd[num])
            cast_list.append(cast_list_even[num])
        cast_list.append(cast_list_odd[-1])
        if cast_list > 30:
            cast_list = cast_list[:30]
        return director_list,writer_list,cast_list
    except:
        return director_list,writer_list,cast_list

In [5]:
def get_review_info(movie_page):
    import requests
    from bs4 import BeautifulSoup
    
    review = None
    if movie_page.find('div',class_='article', id='titleUserReviewsTeaser'):
        review_source = movie_page.find('div',class_='article', id='titleUserReviewsTeaser')
        if review_source.find_all('a'):
            review = ''
            review_source_url = 'https://www.imdb.com' + review_source.find_all('a')[-1].get('href')
            review_sort_url = review_source_url + '?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=10'#按照helpfullness和rating评级选取review
            review_response = requests.get(review_sort_url)
            review_page = BeautifulSoup(review_response.content,'lxml')
            reviews = review_page.find_all('div',class_='text show-more__control')[:3]
            for each_review in reviews:
                each_review = each_review.get_text()
                review += each_review
    return review

In [6]:
def get_movie_info(movie_url_list):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    import re
    
    movie_info_list = []
    for info in movie_url_list:
        movie_name = info[0]
        runtime = info[1]
        genre = info[2]
        movie_url = info[3]
        movie_id = movie_url[-10:-1]
        response = requests.get(movie_url)
        movie_page = BeautifulSoup(response.content,'lxml')
        rate,rating_count = get_titleBar_info(movie_page)
        country,language,release_date,original_title,budget,gross_usa,cumulative_worldwide_gross,production_company = get_titleDetails_info(movie_page,movie_id)
        director,writer,cast = get_cast_and_crew_info(movie_id)
        review = get_review_info(movie_page)
        list_=[movie_name,runtime,genre,movie_url,movie_id,rate,rating_count,country,language,release_date,original_title,director,writer,cast,production_company,budget,gross_usa,cumulative_worldwide_gross,review]
        movie_info_list.append(list_)
    df = pd.DataFrame(movie_info_list,columns=['title','runtime','genre','homepage','imbd id','rate','rating count','country','language','release date','original title','director','writer','cast','production company','budget','gross usa','cumulative worldwide gross','review'])
    return df

##### 1.2 Scrape data by these code below
- for example: All movies in 2000
- return `yyyy_mm.csv`

In [7]:
def scrape_without_collection():
    import pandas as pd
    dates=pd.read_csv(r'dates.csv')
    # print(dates.columns)
    # print()
    start_point=0
    end_point=4
    for i in range(start_point, end_point):
        date=dates.iloc[i]
        page_link=date['page_link'][13:-1]
        file_name=date['file_name']
        number=date['number']
        print(page_link,file_name,number)

        print('scraping: '+page_link)
        movie_url_list = get_movie_url_and_basic_info(page_link,number)
        df = get_movie_info(movie_url_list)
        print('saved: '+ file_name)
    return df

#df= scrape_without_collection()
#df.to_csv(r'./vince/'+file_name+'.csv', index=True)

### 2.Series Movie features
  - return `yyyy_mm_collection.csv`

In [8]:
def Series_movie():
    import requests
    import time
    import tmdbsimple as tmdb
    import numpy as np
    import pandas as pd
    api_key = 'de37cbc8e477b1c8bca247fdaf501c03'
    days_nonleap={'01':['01','31'],'02':['01','28'],'03':['01','31'],'04':['01','30'],'05':['01','31'],'06':['01','30'],'07':['01','31'],'08':['01','31'],'09':['01','30'],'10':['01','31'],'11':['01','30'],'12':['01','31']}
    days_leap={'01':['01','31'],'02':['01','29'],'03':['01','31'],'04':['01','30'],'05':['01','31'],'06':['01','30'],'07':['01','31'],'08':['01','31'],'09':['01','30'],'10':['01','31'],'11':['01','30'],'12':['01','31']}
    start_year=1993
    year=start_year
    end_year=1988
    while year>end_year:
        year=year-1
        if (year%100!=0 and year%4==0) or (year%100==0 and year%400==0):
            days=days_leap
        else:
            days=days_nonleap
        for mm in days:
            print(mm,days[mm])
            movies=list()
            request_number=0
            page=1
            total_page=1
            sleep=1
            limit=4
            while page<=total_page:
                #print(page,' of ',total_page)
                #print('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key+'&page='+str(page)+'&primary_release_date.gte=2019-'+mm+'-'+days[mm][0]+'&primary_release_date.lte=2019-'+mm+'-'+days[mm][1])
                response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key+'&page='+str(page)+'&primary_release_date.gte='+str(year)+'-'+mm+'-'+days[mm][0]+'&primary_release_date.lte='+str(year)+'-'+mm+'-'+days[mm][1])
                #print(response.status_code)
                page+=1
                request_number+=1            # request one more time
                if request_number % limit == 0:
                    time.sleep(sleep)  # Sleep for 3 seconds every tenth requests

                if response.status_code==200:
                    total_page=response.json()['total_pages']

                    results=response.json()['results']
                    attrs=['id']
                    for j in results:
                        #print(j.keys())
                        movie=dict()
                        for k in attrs:
                            if k in j.keys():
                                movie[k]=j[k]  
                            else:
                                movie[k]=''
                        if movie['id']!='':
                            addi_results = requests.get('https://api.themoviedb.org/3/movie/'+str(movie['id'])+'?api_key=' + api_key+'&language=en-US&append_to_response=credits,releases,images')
                            request_number+=1
                            if request_number % limit == 0 and request_number!=0:
                                time.sleep(sleep)  # Sleep for 3 seconds every tenth request

                            if addi_results.status_code==200:
                                addi_results=addi_results.json()
                                addi_attrs=['belongs_to_collection','imdb_id']
                                for m in addi_attrs:
                                    if m in addi_results.keys():
                                        movie[m]=addi_results[m]
                                    else:
                                        movie[m]=''
                            else:
                                continue
                        movies.append(movie)
                else:
                    raise ValueError("Can't get any data:"+str(page)+"th trial")
            movies_pd=pd.DataFrame(movies)
            movies_pd.set_index('id')
            
            return movies_pd
        
#movies_pd= Series_movie()
#movies_pd.to_csv(str(year)+'_'+mm+'_collection.csv')

### 3. Merge all internal features
- Concatenate collection_movies.csv and movie data.csv
- Finally, we have `imdb movies.csv` of all movies from 1930-2019 with 19 internal features

In [9]:
def moviedata():
    import pandas as pd
    import glob
    path =r'collection data sample'
    allFiles = glob.glob(path + "/*.csv")
    movie_data = pd.DataFrame()
    list_ = []

    for file_ in allFiles:
        df = pd.read_csv(file_)
        movie_data = pd.concat((df, movie_data), axis=0)

    del movie_data['Unnamed: 0']
    movie_data.rename(columns={'imbd id':'imdb id'},inplace=True)
    movie_data.drop_duplicates(subset = 'imdb id', keep = 'first', inplace = True) 
    
    return movie_data

# movie_data = moviedata()
# movie_data.to_csv(r'moviedata.csv')

In [10]:
try:
    collection = pd.read_csv('collection_movies.csv')
    info = pd.read_csv('movie data.csv')
    del info['Unnamed: 0']
    collection.dropna()
    collection_dic = {collection['imdb_id'][i]:collection['collection'][i] for i in range(len(collection))}
    imdb_id = list(collection['imdb_id'])
    collection_list = []
    for i in range(len(info)):
        if info['imdb id'][i] in imdb_id:
            collection_list.append(collection_dic[info['imdb id'][i]])
        else:
            collection_list.append(None)
    info['belongs to collection'] = collection_list
    info.to_csv(r'imdb movies.csv')
except:
    pass

### 4.Internal Data Result

In [11]:
import pandas as pd
df= pd.read_csv('movies.csv')
df.head()

,Unnamed: 0,title,imdb id,runtime,genre,rate,rating count,country,language,release date,budget,gross usa,cumulative worldwide gross,director,writer,cast,production company
0,0,To Kill a Mockingbird,tt0056592,129,"['Crime', 'Drama']",8.3,278331.0,['USA'],['English'],1963-03-16,2000000.0,592237.0,599146.0,['nm0612322'],"['nm0497369', 'nm0285210']","['nm0000060', 'nm0576345', 'nm0653942', 'nm061...","['co0021358', 'co0110362', 'co0110361']"
1,1,Lawrence of Arabia,tt0056172,228,"['Adventure', 'Biography', 'Drama']",8.3,249053.0,['UK'],"['English', 'Arabic', 'Turkish']",1962-12-11,15000000.0,45306425.0,45710874.0,['nm0000180'],"['nm0493042', 'nm0004122', 'nm0933858']","['nm0000564', 'nm0000027', 'nm0000063', 'nm037...",['co0103190']
2,2,Dr. No,tt0055928,110,"['Action', 'Adventure', 'Thriller']",7.3,136949.0,['UK'],"['English', 'French']",1962-10-07,1100000.0,16067035.0,16067035.0,['nm0950109'],"['nm0537363', 'nm0367820', 'nm0558435', 'nm000...","['nm0000125', 'nm0000266', 'nm0936476', 'nm052...",['co0054040']
3,3,How the West Was Won,tt0056085,164,['Western'],7.1,17251.0,['USA'],"['English', 'Arapaho']",1963-02-20,15000000.0,76729.0,76729.0,"['nm0000406', 'nm0368871', 'nm0550892', 'nm086...","['nm0916139', 'nm0310775']","['nm0004647', 'nm0002011', 'nm0000020', 'nm042...","['co0007143', 'co0110887']"
4,4,The Manchurian Candidate,tt0056218,126,"['Drama', 'Thriller']",7.9,68378.0,['USA'],['English'],1962-10-24,2200000.0,2757256.0,2757256.0,['nm0001239'],"['nm0174410', 'nm0043480', 'nm0001239']","['nm0000069', 'nm0002131', 'nm0001463', 'nm000...",['co0018343']


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7740 entries, 0 to 7739
Data columns (total 17 columns):
Unnamed: 0                    7740 non-null int64
title                         7740 non-null object
imdb id                       7740 non-null object
runtime                       7740 non-null int64
genre                         7740 non-null object
rate                          7740 non-null float64
rating count                  7740 non-null float64
country                       7740 non-null object
language                      7740 non-null object
release date                  7740 non-null object
budget                        7740 non-null float64
gross usa                     7740 non-null float64
cumulative worldwide gross    7740 non-null float64
director                      7740 non-null object
writer                        7740 non-null object
cast                          7740 non-null object
production company            7740 non-null object
dtypes: float64(5), int

In [13]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')